# 🏆 UEFA Champions League 2025/2026 Prediction Model
## Premium Machine Learning Approach

Welcome to this **advanced predictive analysis** of the Champions League winner. We use a **Machine Learning** approach (Random Forest) combined with **interactive visualizations** to estimate the probability of each team winning the trophy.

### Factors Considered:
- **Squad Quality**: Attack, Midfield, Defense ratings (Performance Indices).
- **Experience**: Number of previous UCL titles.
- **UEFA Coefficient**: Official UEFA 5-year coefficient points (Realism Factor).
- **Financial Power**: Squad Market Value.
- **Current Form**: Weighted performance in the last 15 games.
- **Champions League DNA**: Special adjustment for historic performance.

---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Setup style for beautiful static plots
sns.set_theme(style="darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 1. Load Data
We load the dataset containing the statistics for the top 32 teams competing in the 2025/2026 season.

In [ ]:
# Load the dataset
df = pd.read_csv('data.csv')

# Display the top 5 contenders based on Current Form
display(df.sort_values(by='Current_Form', ascending=False).head(10))

## 2. Exploratory Data Analysis (EDA)
Understanding the landscape of the competition before predicting.

### 2.1 Correlation Matrix
Which factors are most correlated with a high Squad Value and Ratings? This helps us see if money correlates with attack/defense (usually yes).

In [ ]:
# Select numerical columns for correlation
numerical_cols = ['Attack_Rating', 'Midfield_Rating', 'Defense_Rating', 'Squad_Value_M', 
                  'UEFA_Coefficient', 'Current_Form', 'Manager_UCL_Experience']

corr_matrix = df[numerical_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix of Team Stats')
plt.show()

### 2.2 Squad Value vs. Overall Team Rating
Comparing the financial power of the top teams.

In [ ]:
top_10_value = df.sort_values(by='Squad_Value_M', ascending=False).head(10)

fig = px.bar(top_10_value, x='Squad_Value_M', y='Team', orientation='h', 
             title='Top 10 Most Valuable Squads (Millions €)',
             labels={'Squad_Value_M': 'Market Value (€M)'},
             color='Squad_Value_M', color_continuous_scale='Viridis')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

### 2.3 Interactive Radar Chart: FC Barcelona vs Rivals
Let's compare **FC Barcelona**, **Real Madrid**, and **Arsenal** (Current Favorites) directly on key attributes using an interactive chart.

In [ ]:
def create_radar_chart_plotly(df, teams, features):
    fig = go.Figure()

    for team in teams:
        # Get values for the team
        subset = df[df['Team'] == team][features].values.flatten().tolist()
        subset += [subset[0]] # Close the loop
        
        fig.add_trace(go.Scatterpolar(
            r=subset,
            theta=features + [features[0]],
            fill='toself',
            name=team
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[60, 100]  # Zoom in to see differences
            )),
        showlegend=True,
        title='Team Comparison: Top Contenders (Interactive)',
        height=600
    )
    fig.show()

# Compare Barca with the two other "titans"
create_radar_chart_plotly(df, ['FC Barcelona', 'Real Madrid', 'Arsenal'], 
                   ['Attack_Rating', 'Midfield_Rating', 'Defense_Rating', 'Current_Form', 'UEFA_Coefficient'])

## 3. Machine Learning Model Training

Since we are predicting a future event (2026 Winner), we don't have "ground truth" labels for this specific season yet. 

**Strategy**: We will generate a **Synthetic Historical Dataset** representing past champions and non-champions from the last 20 years. We will train a **Random Forest Classifier** on this data to learn the characteristics of a "Winner" (e.g., high defense, high squad value, experienced manager), and then apply this model to our current 2026 data.

In [ ]:
# --- Sythnetic Training Data (Simulating past 20 years of UCL data) ---
np.random.seed(42)

# Create 1000 dummy team-seasons
n_samples = 1000

# Generate random features resembling our data structure
train_attack = np.random.normal(80, 10, n_samples)
train_midfield = np.random.normal(80, 10, n_samples)
train_defense = np.random.normal(80, 10, n_samples)
train_value = np.random.normal(500, 250, n_samples)
train_exp = np.random.normal(70, 15, n_samples) # This simulates UEFA Coefficient now
train_form = np.random.normal(7.5, 1.5, n_samples)
train_manager = np.random.randint(1, 10, n_samples)

X_train_synthetic = pd.DataFrame({
    'Attack_Rating': train_attack,
    'Midfield_Rating': train_midfield,
    'Defense_Rating': train_defense,
    'Squad_Value_M': train_value,
    'UEFA_Coefficient': train_exp,
    'Current_Form': train_form,
    'Manager_UCL_Experience': train_manager
})

# Define a "Winning Formula" to label the data (The "Target" variable)
# UPDATED LOGIC v2: HYPER-FOCUS ON CURRENT FORM
score = (
    0.15 * train_attack +
    0.1 * train_defense +    
    0.1 * train_midfield +
    0.05 * (train_value / 10) + 
    0.05 * train_exp +       
    0.55 * (train_form * 10) # Current Form >50% impact
)

# Add some randomness/noise (upsets happen!)
score += np.random.normal(0, 10, n_samples)

# Top 5% percent are marked as "Winners" (1), others Losers (0)
threshold = np.percentile(score, 95)
y_train_synthetic = (score > threshold).astype(int)

print(f"Training data created with {n_samples} samples. Positive class (Winners): {sum(y_train_synthetic)}")

In [ ]:
# Initialize and Train Random Forest
rf_model = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=5)
rf_model.fit(X_train_synthetic, y_train_synthetic)

# Check Feature Importance
importances = rf_model.feature_importances_
feature_names = X_train_synthetic.columns

fig = px.bar(x=importances, y=feature_names, orientation='h', 
             title='Feature Importance: What makes a CL Winner?',
             labels={'x': 'Importance Score', 'y': 'Feature'},
             color=importances, color_continuous_scale='Magma')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

## 4. Prediction: Who will win in 2026?
We now apply our trained model to the real 2026 dataset.

In [ ]:
# Prepare 2026 data for prediction (select same columns)
X_2026 = df[feature_names]

# Predict Probabilities (we want the probability of being class 1 'Winner')
probs = rf_model.predict_proba(X_2026)[:, 1]

# Add to dataframe
df['Win_Probability'] = probs

# Normalize probablities so they sum to ~100% (roughly, for visualization)
df['Win_Probability_Normalized'] = (df['Win_Probability'] / df['Win_Probability'].sum()) * 100

# Sort by probability
results = df[['Team', 'Win_Probability_Normalized', 'Attack_Rating', 'Defense_Rating', 'Domestic_League_Rank']].sort_values(by='Win_Probability_Normalized', ascending=False)

# Display Top 10 Candidates
display(results.head(10))

### 4.1 Visualizing the Favorites

In [ ]:
fig = px.bar(results.head(10), x='Win_Probability_Normalized', y='Team', orientation='h', 
             title='Predicted Champions League Winner 2026',
             labels={'Win_Probability_Normalized': 'Estimated Probability of Winning (%)'},
             color='Win_Probability_Normalized', 
             color_continuous_scale='Bluered')

fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

## 5. Final Verdict: Who will lift the trophy? 🏆

Based on the AI model's simulation, here is the official winner prediction.

In [ ]:
top_team = results.iloc[0]
name = top_team['Team']
prob = top_team['Win_Probability_Normalized']

print(f"⭐ CHAMPION 2026 PREDICTION ⭐")
print(f"The model predicts {name.upper()} as the winner!")
print(f"Winning Probability: {prob:.2f}%")

runner_up = results.iloc[1]
print(f"\n🥈 Runner-up: {runner_up['Team']} ({runner_up['Win_Probability_Normalized']:.2f}% chance)")

print(f"\nWhy {name}? They have the optimal balance of Squad Value, Form, and Coefficients according to the model trained on historical data.")